In [1]:
'''

LLMs with function calling 

'''

import os 
os.environ["OPENAI_API_KEY"] = ""

import nest_asyncio
nest_asyncio.apply()

In [2]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from openai.types.chat import ChatCompletionMessageToolCall

In [3]:
# simple function tool

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)

In [4]:
'''

Agent - Mainly comprises three things - 
1) Call LLM to decide which tool to use and with what arguments 
2) Call the tool with the arguments to obtain an output
3) Call OpenAI to synthesize a response from the conversation context and the tool output.

'''

class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    # reset simply resets the context of the messages/conversation 
    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(
        self, tool_call: ChatCompletionMessageToolCall
    ) -> ChatMessage:
        id_ = tool_call.id
        function_call = tool_call.function
        tool = self._tools[function_call.name]
        output = tool(**json.loads(function_call.arguments))
        return ChatMessage(
            name=function_call.name,
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call.name,
            },
        )

In [5]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [6]:
agent.chat("Hi")

'Hello! How can I assist you today?'

In [7]:
agent.chat("What is 2123 * 215123")

'The product of 2123 multiplied by 215123 is 456,706,129.'

In [8]:
# Better Function 

from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405

(121 * 3) + 42 is equal to 405.


In [9]:
# other type of functions 
def call_police() -> str:
    """Calls police in danger/problem/assitance"""
    return "Keep the person calm and tell police has been notified"

def call_emergency_contacts() -> str:
    """Calls emergency contact person"""
    return "Keep the person calm and tell your emergency people have been notified"

def call_ambulance() -> str:
    """Calls ambulance in need of help"""
    return "Keep the person calm and tell ambulance is nearby"


police_tool = FunctionTool.from_defaults(fn = call_police)
emergency_tool = FunctionTool.from_defaults(fn = call_emergency_contacts)
ambulance_tool = FunctionTool.from_defaults(fn = call_ambulance)


llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [police_tool, emergency_tool, ambulance_tool], llm=llm, verbose=True
)



In [10]:
response = agent.chat("I got caught up in an accident, my leg hurts. Help!")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Help!
=== Calling Function ===
Calling function: call_ambulance with args: {}
Got output: Keep the person calm and tell ambulance is nearby

I'm sorry to hear that you've been in an accident and that your leg hurts. I've called an ambulance for you. They should be arriving shortly. In the meantime, try to stay calm and avoid moving too much. Help is on the way.


In [11]:
response = agent.chat("I got caught up in an accident, my leg hurts. Inform police, Help!")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Inform police, Help!
=== Calling Function ===
Calling function: call_police with args: {}
Got output: Keep the person calm and tell police has been notified

I'm sorry to hear that you've been in an accident and that your leg hurts. I've notified the police about the situation. They should be on their way to assist you. In the meantime, try to stay calm and avoid moving too much. Help is on the way.


In [12]:
response = agent.chat("I got caught up in an accident, my leg hurts I need assistance. Inform police the guy is running away, Help!")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts I need assistance. Inform police the guy is running away, Help!
=== Calling Function ===
Calling function: call_police with args: {}
Got output: Keep the person calm and tell police has been notified

I'm sorry to hear that you've been in an accident and that your leg hurts. I've notified the police about the situation, including the fact that the other person involved is running away. They will be on their way to assist you and apprehend the person. In the meantime, try to stay calm and avoid moving too much. Help is on the way.


In [13]:
# other type of functions 
def call_police(message:str) -> str:
    """Calls police in danger/problem/assitance and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell police has been notified and see if other tools need to be used"

def call_emergency_contacts(message:str) -> str:
    """Calls emergency contact person and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell your emergency people have been notified and see if other tools need to be used"

def call_ambulance(message:str) -> str:
    """Calls ambulance in need of help and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell ambulance is nearby and see if other tools need to be used"


def print_message(msg:str):
    print("Message received " + msg)


police_tool = FunctionTool.from_defaults(fn = call_police)
emergency_tool = FunctionTool.from_defaults(fn = call_emergency_contacts)
ambulance_tool = FunctionTool.from_defaults(fn = call_ambulance)


llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [police_tool, emergency_tool, ambulance_tool], llm=llm, verbose=True
)



In [14]:
response = agent.chat("I got caught up in an accident, my leg hurts. Help!")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Help!
=== Calling Function ===
Calling function: call_ambulance with args: {
  "message": "I got caught up in an accident, my leg hurts. Please send an ambulance!"
}
Message received I got caught up in an accident, my leg hurts. Please send an ambulance!
Got output: Keep the person calm and tell ambulance is nearby and see if other tools need to be used

I have called an ambulance for you. They will be arriving shortly to assist you. Please try to stay calm and wait for their arrival. If you have any other injuries or need any other assistance, please let me know.


In [17]:
# async chat 

import time

# other type of functions 
def call_police(message:str) -> str:
    """Calls police in danger/problem/assitance and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell police has been notified and see if other tools need to be used"

def call_emergency_contacts(message:str) -> str:
    """Calls emergency contact person and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell your emergency people have been notified and see if other tools need to be used"

def call_ambulance(message:str) -> str:
    """Calls ambulance in need of help and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell ambulance is nearby and see if other tools need to be used"


def print_message(msg:str):
    time.sleep(10)
    print("Message received " + msg)


police_tool = FunctionTool.from_defaults(fn = call_police)
emergency_tool = FunctionTool.from_defaults(fn = call_emergency_contacts)
ambulance_tool = FunctionTool.from_defaults(fn = call_ambulance)


llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [police_tool, emergency_tool, ambulance_tool], llm=llm, verbose=True
)



response = await agent.achat("I got caught up in an accident, my leg hurts. Help!")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Help!
=== Calling Function ===
Calling function: call_ambulance with args: {
  "message": "I got caught up in an accident, my leg hurts. Please send an ambulance!"
}
Message received I got caught up in an accident, my leg hurts. Please send an ambulance!
Got output: Keep the person calm and tell ambulance is nearby and see if other tools need to be used

I have called an ambulance for you. They will be arriving shortly to assist you. Please try to stay calm and wait for their arrival. If there are any other tools or equipment that you need in the meantime, please let me know.


In [22]:
# with a prompt
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [police_tool, emergency_tool, ambulance_tool],
    llm = llm,
    verbose = True,
    system_prompt = "You are an emergency responder, which provide quick and efficient assitant to the persone in need by calling police, emergency contacts or ambulance",
)


response = agent.chat("I got caught up in an accident, my leg hurts. Help!")
print(response)

Added user message to memory: I got caught up in an accident, my leg hurts. Help!
=== Calling Function ===
Calling function: call_emergency_contacts with args: {
  "message": "Emergency! I got caught up in an accident and my leg hurts. Please send help!"
}
Message received Emergency! I got caught up in an accident and my leg hurts. Please send help!
Got output: Keep the person calm and tell your emergency people have been notified and see if other tools need to be used

I have notified your emergency contacts about the accident and your leg injury. Help is on the way. In the meantime, try to stay calm and avoid moving too much to prevent further injury.


In [24]:
# forcing to use a particular function 
'''

We can use 'tool_choice' as operator to decide which function/tool to use

'''
def call_police(message:str) -> str:
    """Calls police in danger/problem/assitance and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell police has been notified and see if other tools need to be used"

def call_emergency_contacts(message:str) -> str:
    """Calls emergency contact person and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell your emergency people have been notified and see if other tools need to be used"

def call_ambulance(message:str) -> str:
    """Calls ambulance in need of help and inform them about the situation"""
    print_message(message)
    return "Keep the person calm and tell ambulance is nearby and see if other tools need to be used"


def print_message(msg:str):
    print("Message received -- " + msg)


police_tool = FunctionTool.from_defaults(fn = call_police)
emergency_tool = FunctionTool.from_defaults(fn = call_emergency_contacts)
ambulance_tool = FunctionTool.from_defaults(fn = call_ambulance)


llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [police_tool, emergency_tool, ambulance_tool], llm=llm, verbose=True
)



response = agent.chat("I got caught up in an accident, my leg hurts. Help!", tool_choice="auto")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Help!
=== Calling Function ===
Calling function: call_ambulance with args: {
  "message": "I got caught up in an accident, my leg hurts. Please send an ambulance!"
}
Message received -- I got caught up in an accident, my leg hurts. Please send an ambulance!
Got output: Keep the person calm and tell ambulance is nearby and see if other tools need to be used

I have called an ambulance for you. They will be arriving shortly to assist you. Please try to stay calm and wait for their arrival. If there are any other tools or equipment that you need in the meantime, please let me know.


In [27]:
response = agent.chat("I got caught up in an accident, my leg hurts. Inform my emergency contact Help!", tool_choice="call_police")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Inform my emergency contact Help!
=== Calling Function ===
Calling function: call_police with args: {
  "message": "I got caught up in an accident, my leg hurts. Please inform my emergency contact!"
}
Message received -- I got caught up in an accident, my leg hurts. Please inform my emergency contact!
Got output: Keep the person calm and tell police has been notified and see if other tools need to be used

=== Calling Function ===
Calling function: call_emergency_contacts with args: {
  "message": "I got caught up in an accident, my leg hurts. Please help!"
}
Message received -- I got caught up in an accident, my leg hurts. Please help!
Got output: Keep the person calm and tell your emergency people have been notified and see if other tools need to be used

I have informed your emergency contact about the accident and your current situation. They will be notified and will provide assistance as needed. Please tr

In [28]:
# forcing not to call anyfunction 
response = agent.chat("I got caught up in an accident, my leg hurts. Help!", tool_choice="none")
print(str(response))

Added user message to memory: I got caught up in an accident, my leg hurts. Help!
I'm sorry to hear that you've been in an accident and that your leg hurts. I'll do my best to assist you. 

First, I recommend calling emergency services immediately. They will be able to provide the necessary medical assistance and transport you to the hospital if needed. If you're unable to make the call yourself, please let me know, and I'll assist you in contacting the appropriate authorities.

In the meantime, try to stay as still as possible and avoid putting weight on your injured leg. If you have any first aid supplies nearby, such as bandages or ice packs, you can use them to help alleviate pain and reduce swelling.

Please remember that I am an AI assistant and cannot provide direct medical assistance. It's important to seek professional medical help as soon as possible.
